In [1]:
import pandas as pd
df = pd.read_csv("train_data.csv")
df

,id,ref,year,year_ref,doc_type,n_citation_ref,n_citation,n_citation_ref_count,Point,citations_per_year
0,1091,2005687710,0.000000,0.000000,none,0.000347,0.000290,3,32,0.000000
1,1674,1535888970,0.992075,0.992075,Conference,0.010745,0.000290,12,2662,0.009357
2,1674,1992876689,0.000000,0.000000,none,0.000693,0.000290,5,52,0.000000
3,1674,1993710814,0.000000,0.000000,none,0.001560,0.000290,10,102,0.000000
4,1674,2035653341,0.000000,0.000000,none,0.001906,0.000290,12,122,0.000000
...,...,...,...,...,...,...,...,...,...,...
303168,164734591,1996793998,0.000000,0.000000,none,0.000000,0.000580,1,14,0.000000
303169,164736972,117400678,0.000000,0.000000,none,0.000000,0.001159,1,18,0.000000
303170,164736972,188145618,0.994056,0.994056,Conference,0.042808,0.001159,54,4558,0.045501
303171,164736972,1514298587,0.000000,0.000000,none,0.000693,0.001159,5,58,0.000000


In [13]:
import pandas as pd
df_final = pd.read_csv('point_delete_ref0.csv')

In [ ]:
df = pd.read_csv("train_data.csv")
df

# Tính point

In [5]:
# Function to calculate the point
def calculate_point(row, related_refs):
    base_point = 0
    base_point = row['year_ref'] * 100 + row['n_citation_ref'] * 100 + row['n_citation']*100 + row['citations_per_year'] * 200

    # Add 200 points if the ref is in the list of related references
    if row['ref'] in related_refs:
        base_point += 200

    return int(base_point)

# Example input references that you want to prioritize
related_refs = [2005687710, 2018037215]  # Modify this list based on your input

# Applying the function to calculate points
df['Point'] = df.apply(calculate_point, related_refs=related_refs, axis=1)


In [2]:
# Group by 'id' and aggregate the references and points
result = (
    df.groupby('id')
    .agg({
        'ref': lambda x: ' '.join(map(str, x)),  # Use space instead of comma
        'Point': lambda x: f"{x.sum()}:{' '.join(map(str, x))}"  # Format with space
    })
    .reset_index()
)

# Prepare the output format
output_lines = [
    f"{row['ref']}:{row['Point']}" for _, row in result.iterrows()
]

# Save to a .txt file
with open('train_data.txt', 'w') as file:
    for line in output_lines:
        file.write(line + '\n')

print("Data has been saved to 'train_data.txt'.")

Data has been saved to 'train_data.txt'.


# Sau khi chạy EIHI được kết quả -> search

In [14]:
# Filter the DataFrame for the specified references
target_refs = [1480600454, 1556566737]
filtered_articles = df_final[df_final['ref'].isin(target_refs)]

In [15]:
filtered_articles

,id,ref,year,year_ref,doc_type,n_citation_ref,n_citation,n_citation_ref_count,Point
353,59226,1556566737,2004,2004,Conference,3377,4,2336,35818
2869,493438,1480600454,0,0,none,856,7,856,8574
3383,598789,1556566737,2004,2004,Conference,3377,4,2336,35818
4948,852384,1556566737,2004,2004,Conference,3377,8,2336,35826
7744,1362128,1556566737,2004,2004,Conference,3377,14,2336,35838
...,...,...,...,...,...,...,...,...,...
949251,161537417,1480600454,0,0,none,856,23,856,8606
949256,161537417,1556566737,2004,2004,Conference,3377,23,2336,35856
953610,162318746,1480600454,0,0,none,856,26,856,8612
956471,162822354,1556566737,2004,2004,Conference,3377,1,2336,35812


In [16]:
target_refs = [1480600454, 1556566737]
related_refs = [2005687710, 2018037215]

# Filter the DataFrame for the specified target references
filtered_articles = df[df['ref'].isin(target_refs)]

# Count the total number of filtered articles
total_filtered = filtered_articles.shape[0]

# Count the number of filtered articles that are also in related_refs
count_related = filtered_articles[filtered_articles['ref'].isin(related_refs)].shape[0]

# Calculate the percentage
percentage = (count_related / total_filtered * 100) if total_filtered > 0 else 0

# Display the results
print(f'Total filtered articles: {total_filtered}')
print(f'Count of related references in filtered articles: {count_related}')
print(f'Percentage of related references in filtered articles: {percentage:.2f}%')


Total filtered articles: 639
Count of related references in filtered articles: 0
Percentage of related references in filtered articles: 0.00%


In [1]:
import pandas as pd

# Đọc dữ liệu bài báo từ file CSV
df = pd.read_csv(r"E:\tlcn\HUI_GNN\hui\data\citation_delete_doi\hui_citation\basic_data\citation_network.csv")

# Tạo một từ điển với id làm khóa và title làm giá trị
id_to_title = dict(zip(df['id'], df['title']))

# Đọc dữ liệu từ file train.txt
with open('outputEIHI_200.txt', 'r') as file:
    rows = file.readlines()

# Tạo danh sách để lưu kết quả
results = []

# Khớp id với title và giữ lại thông tin UTIL
for row in rows:
    parts = row.strip().split()  # Tách các phần trong dòng
    util_index = parts.index('#UTIL:')  # Xác định vị trí của UTIL
    ids = parts[:util_index]  # Lấy danh sách ID
    util = parts[util_index + 1]  # Lấy giá trị UTIL

    # Khớp các id với title
    titles = [id_to_title.get(int(id), "Title không tìm thấy") for id in ids]
    
    # Định dạng kết quả
    formatted_result = "\n".join([f"ID: {id} - Title: {title}" for id, title in zip(ids, titles)])
    results.append(f"{formatted_result}\nUTIL: {util}\n")

# In kết quả theo cách dễ nhìn
for result in results:
    print(result)


C:\Users\ngchn\AppData\Local\Temp\ipykernel_4248\4070370723.py:4: DtypeWarning: Columns (0,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"E:\tlcn\HUI_GNN\hui\data\citation_delete_doi\hui_citation\basic_data\citation_network.csv")


ID: 2091257550 - Title: Title không tìm thấy
ID: 2154010459 - Title: Title không tìm thấy
UTIL: 962512

ID: 2057012437 - Title: Title không tìm thấy
ID: 2100379340 - Title: Title không tìm thấy
UTIL: 1088202

ID: 1556566737 - Title: Title không tìm thấy
ID: 1842017652 - Title: Memory Efficient Data Structures for Explicit Verification of Timed Systems
UTIL: 993102

ID: 1556566737 - Title: Title không tìm thấy
ID: 2101508170 - Title: Title không tìm thấy
UTIL: 2203216

ID: 1556566737 - Title: Title không tìm thấy
ID: 2194223632 - Title: Title không tìm thấy
UTIL: 2666446

ID: 1556566737 - Title: Title không tìm thấy
ID: 2132524091 - Title: Title không tìm thấy
UTIL: 1295164

ID: 1556566737 - Title: Title không tìm thấy
ID: 1594381772 - Title: Formalising java's data race free guarantee
UTIL: 1409336

ID: 1556566737 - Title: Title không tìm thấy
ID: 1606555018 - Title: Monodic fragments of first-order temporal logics: 2000-2001 A.D
UTIL: 2360482

ID: 1556566737 - Title: Title không tìm t